In [221]:
from imports import *
from catalog_object import *
import mwdust, estimate_PDF
from scipy.interpolate import LinearNDInterpolator as lint
%matplotlib inline

## get pre-GAIA data on Kepler M dwarfs with planets

Read in data from the NASA Exoplanet Archive and cross-match with 2MASS data to get photometric uncertainties.

In [196]:
# get pre-GAIA data on Kepler planet hosts
cols = (1,5,6,7,14,15,16,20,21,22,23,24,25,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52)
fname = 'Keplertargets/NASAarchive_confirmed_Keplerlowmassstars.csv'
d = np.loadtxt(fname, delimiter=',', skiprows=66, usecols=cols).T
KepIDs,Ps,ehiP,eloP,D,ehiD,eloD,rpRs,ehirpRs,elorpRs,rps,ehirp,elorp,Teff,ehiTeff,eloTeff,logg,ehilogg,elologg,FeH,ehiFeH,eloFeH,Rss,ehiRs,eloRs,Mss,ehiMs,eloMs,ras,decs,Kepmag,Jmag,Hmag,Kmag = d
N = KepIDs.size
print N

280


In [197]:
# match to 2MASS sources to get photometric uncertainties
ra2,dec2,_,Jmag2,_,eJmag2,Hmag2,_,eHmag2,Kmag2,_,eKmag2 = np.load('Keplertargets/fp_2mass.fp_psc12298.npy').T
eJmag, eHmag, eKmag = np.zeros(N), np.zeros(N), np.zeros(N)
for i in range(N):
    
    # search within 10 arcsec
    sep = np.sqrt((ra2 - ras[i])**2 + (dec2 - decs[i])**2)
    close = sep*3600 < 10
    
    # find photometric matches
    jmag, hmag, kmag = Jmag[i], Hmag[i], Kmag[i]
    if np.any((Jmag2[close] == jmag) & (Hmag2[close] == hmag) & (Kmag2[close] == kmag)):  # exact match
        g = (sep*3600 < 10) & (Jmag2 == jmag) & (Hmag2 == hmag) & (Kmag2 == kmag)
        eJmag[i] = eJmag2[g]
        eHmag[i] = eHmag2[g]
        eKmag[i] = eKmag2[g]
        
    else:
        print i

In [199]:
# check for missing photometric uncertainties and replace those values
def replace_bad_emag(mags2MASS, emags2MASS, mags, emags, mag_range=.3):
    assert mags.size == emags.size
    print 'There are %i/%i bad photometric uncertainies.'%(np.isnan(emags).sum(), mags.size)
    for i in np.arange(mags.size, dtype=int)[np.isnan(emags)]:
        g = (mags2MASS >= mags[i]-mag_range) & (mags2MASS <= mags[i]+mag_range)
        emags[i] = np.nanmedian(emags2MASS[g])
    return emags
    
eJmag = replace_bad_emag(Jmag2, eJmag2, Jmag, eJmag)
eHmag = replace_bad_emag(Hmag2, eHmag2, Hmag, eHmag)
eKmag = replace_bad_emag(Kmag2, eKmag2, Kmag, eKmag)

There are 0/280 bad photometric uncertainies.
There are 0/280 bad photometric uncertainies.
There are 0/280 bad photometric uncertainies.


## get GAIA data on Kepler M dwarfs with planets

Cross-match Kepler M dwarf with planets to GAIA DR2 and save parallaxes, photometry, and positions

In [200]:
# get GAIA data from Megan Bedell's Kepler-GAIA catalog
hdu = fits.open('Keplertargets/kepler_dr2_4arcsec.fits')[1]

hasGAIA = np.zeros(N, dtype=bool)
pars, epars = np.zeros(N), np.zeros(N)
GBPmags, eGBPmags = np.zeros(N), np.zeros(N)
GRPmags, eGRPmags = np.zeros(N), np.zeros(N)
ras, decs = np.zeros(N), np.zeros(N)
ls, bs = np.zeros(N), np.zeros(N)
for i in range(N):
    
    g = hdu.data['kepid'] == KepIDs[i]
    if g.sum() == 0:   # no match
        pars[i], epars[i] = np.repeat(np.nan, 2)
        GBPmags[i], eGBPmags[i] = np.repeat(np.nan, 2)
        GRPmags[i], eGRPmags[i] = np.repeat(np.nan, 2)
        ras[i], decs[i] = np.repeat(np.nan, 2)
        ls[i], bs[i] = np.repeat(np.nan, 2)
        
    elif g.sum() >= 1:  # at least one match (take the first entry if multiple matches)
        index = 0
        hasGAIA[i] = True
        pars[i], epars[i] = hdu.data['parallax'][g][index], hdu.data['parallax_error'][g][index]

        GBPmags[i] = hdu.data['phot_bp_mean_mag'][g][index]            
        FBP = hdu.data['phot_bp_mean_flux'][g][index]
        eFBP = hdu.data['phot_bp_mean_flux_error'][g][index]
        eGBPmags[i] = -2.5*np.log10(FBP / (FBP+eFBP))
        
        GRPmags[i] = hdu.data['phot_rp_mean_mag'][g][index]            
        FRP = hdu.data['phot_rp_mean_flux'][g][index]
        eFRP = hdu.data['phot_rp_mean_flux_error'][g][index]
        eGRPmags[i] = -2.5*np.log10(FRP / (FRP+eFRP))
        
        ras[i], decs[i] = hdu.data['ra'][g][index], hdu.data['dec'][g][index]
        ls[i], bs[i] = hdu.data['l'][g][index], hdu.data['b'][g][index]

## compute distance posteriors (from Bailor-Jones)

In [201]:
# get distance posteriors from Bailor-Jones R scripts
cwd = os.getcwd()
os.chdir('GaiaDistances/')
prefix = 'KepID'
overwrite = False
for i in range(N):
       
    cmd_prefix = 'Rscript get_dist_post.R %s'%prefix
    cmd = '%s_%i %.6e %.6e %.6f %.6f'%(cmd_prefix, KepIDs[i], pars[i], epars[i], ls[i], bs[i])

    # if posterior doesn't exist, compute it using the R script
    fout = 'DistancePosteriors/%s_%i.csv'%(prefix, KepIDs[i])
    if ((not os.path.exists(fout)) or overwrite) and (hasGAIA[i]):
        os.system(cmd)

os.chdir(cwd)

In [202]:
# save distance point estimates
Nsamp = 1000
dists, ehidists, elodists = np.zeros(N), np.zeros(N), np.zeros(N)
mus, ehimus, elomus = np.zeros(N), np.zeros(N), np.zeros(N)
for i in range(N):
    
    if hasGAIA[i]:
        # get distance posterior
        fname = 'GaiaDistances/DistancePosteriors/%s_%i.csv'%(prefix, KepIDs[i])
        distarr, probarr = np.loadtxt(fname, delimiter=',', skiprows=1, usecols=(1,2)).T
        samp_dist = np.random.choice(distarr, Nsamp, p=probarr/probarr.sum()) + np.random.randn(Nsamp)*1e-2

        # save distance point estimates
        v = np.percentile(samp_dist, (16,50,84))
        dists[i], ehidists[i], elodists[i] = v[1], v[2]-v[1], v[1]-v[0]
    
        # sample and save distance modulus
        samp_mu = 5*np.log10(samp_dist) - 5
        v = np.percentile(samp_mu, (16,50,84))
        mus[i], ehimus[i], elomus[i] = v[1], v[2]-v[1], v[1]-v[0]
        
    else:
        dists[i], ehidists[i], elodists[i] = np.repeat(np.nan, 3)
        mus[i], ehimus[i], elomus[i] = np.repeat(np.nan, 3)

## estimate extinction using mwdust

In [214]:
def _compute_AK_mwdust(ls, bs, dist, edist, eAK_frac=.3, RV=3.1):
    '''Using the EB-V map from 2014MNRAS.443.2907S and the extinction vector
    RK = 0.31 from Schlafly and Finkbeiner 2011 (ApJ 737, 103). 
    Where does RV=3.1 come from? See the MIST BC tables where RV is fixed to 3.1'''
    dustmapK = mwdust.Combined15(filter='2MASS Ks')
    dustmapV = mwdust.Combined15(filter=None)  # returns E(B-V) rather than A_lambda
    dist_kpc, edist_kpc = np.ascontiguousarray(dist)*1e-3, \
                          np.ascontiguousarray(edist)*1e-3
    ls, bs = np.ascontiguousarray(ls), np.ascontiguousarray(bs)
    AK, eAK = np.zeros(ls.size), np.zeros(ls.size)
    AV = np.zeros(ls.size)
    for i in range(ls.size):
        v = dustmapK(ls[i], bs[i],
                     np.array([dist_kpc[i], dist_kpc[i]+edist_kpc[i]]))
        AK[i], eAK[i] = v[0], np.sqrt(abs(np.diff(v))**2 + (eAK_frac*v[0])**2)
        v = dustmapV(ls[i], bs[i],
                     np.array([dist_kpc[i], dist_kpc[i]+edist_kpc[i]]))
        AV[i] = v[0]*RV
    return AK, eAK, AV


def MAD(arr):
    return np.median(abs(arr - np.median(arr)))
    

AKs, eAKs, AVs = np.zeros(N), np.zeros(N), np.zeros(N)
for i in range(N):
    
    if hasGAIA[i]:

        # get distance posterior
        fname = 'GaiaDistances/DistancePosteriors/KepID_%i.csv'%KepIDs[i]
        distarr, probarr = np.loadtxt(fname, delimiter=',', skiprows=1, usecols=(1,2)).T
        samp_dist = np.random.choice(distarr, Nsamp, p=probarr/probarr.sum()) + np.random.randn(Nsamp)*1e-2
        
        # compute K-band extinction
        AKs[i], eAKs[i], AVs[i] = _compute_AK_mwdust(ls[i], bs[i], dists[i], MAD(samp_dist))

## setup functions to compute bolometric corrections

In [217]:
def _interpolate_BCK(Teff, e_Teff, logg, FeH, Av):
    '''Interpolate the MIST bolometric correction grids to get a star's K-band bolometric correction.'''
    # select a metallicity grid
    FeHgrid = np.array([-4,-3.5,-3,-2.75,-2.5,-2.25,-2,-1.75,-1.5,-1.25,-1,-.75,-.5,-.25,0,.25,.5,.75])
    inds, coeffs = _get_interpolation_coeffs(FeHgrid, FeH)
    if np.any(coeffs == 0):
        FeHs = FeHgrid[inds[coeffs>0]]
        coeffs = np.ones(1)
    else:
        FeHs = FeHgrid[inds]
    
    # interpolate over remaining parameters for all metallicity grids and with sampled Teff
    BCKs = np.zeros((FeHs.size, 2))
    for i in range(FeHs.size):
        
        # get BC grid for a fixed Fe/H
        label = 'p' if FeHs[i] >= 0 else 'm'
        fname = 'UBVRIplus/feh%s%.3d.UBVRIplus'%(label, abs(FeHs[i])*1e2)
        Teffgrid, logggrid, Avgrid, BCKgrid  = np.loadtxt(fname, skiprows=5, usecols=(0,1,3,12)).T
        
        # compute a separate BCK for Teff across its 1 sigma uncertainty to estimate the uncertainty in BCK
        # because it is dominated by uncertainies in Teff which have the strongest effect on the stellar SED
        BCKs_Teff, j = np.zeros(0), 0
        for t in Teff+np.arange(2)*e_Teff:
            # interpolate to get BCK
            lint_BCK = lint(np.array([Teffgrid,logggrid,Avgrid]).T, BCKgrid)
            BCKs[i,j] = np.append(BCKs_Teff, float(lint_BCK(t, logg, Av)) * coeffs[i])
            j += 1
            
    # sum contributions from each Fe/H
    BCKs = np.sum(BCKs, axis=0)
    assert BCKs.size == 2
    BCK, e_BCK = BCKs[0], abs(float(np.diff(BCKs)))
    return BCK, e_BCK

                     
def _get_interpolation_coeffs(arr, val):
    if val < arr.min():
        return np.argsort(arr)[:1], np.ones(1)
    elif val > arr.max():
        return np.argsort(arr)[-1:], np.ones(1)
    else:
        edgeinds = np.argsort(abs(arr-val))[:2]
        edgevals = arr[edgeinds]
        diff = abs(float(np.diff(edgevals)))
        c1 = abs(val-edgevals[0]) / diff
        coeffs = np.array([1-c1, c1]).reshape(2)
        return edgeinds, coeffs

## compute new stellar parameters

In [223]:
def _sample_Rs_from_MK_Mdwarfs(samp_MK):
    '''Use relation from Mann+2015 (table 1) for M dwarfs'''
    if (np.median(samp_MK) > 4.6) & (np.median(samp_MK) < 9.8):
        a, b, c, Rs_sigma_frac = 1.9515, -.3520, .01680, .0289
        p = np.poly1d((c,b,a))
        samp_MK_tmp = np.copy(samp_MK)
        #samp_MK_tmp[(samp_MK<=4.6) | (samp_MK>=9.8)] = np.nan
        samp_Rs = p(samp_MK_tmp)
        samp_Rs += np.random.normal(0, samp_Rs*Rs_sigma_frac, samp_MK.size)
    else:
        samp_Rs = np.zeros_like(samp_MK) + np.nan
    return samp_Rs


def _sample_Rs_from_MK_Kdwarfs(samp_MK, samp_BCK, samp_Teff):
    if (np.median(samp_MK) < 4.6):
        samp_Mbol = samp_MK + samp_BCK
        samp_L = 3.0128e28 * 10**(-.4*samp_Mbol)  # Watts
        sigma = 5.67e-8
        samp_Rs = rvs.m2Rsun(np.sqrt(samp_L / (4*np.pi*sigma*samp_Teff**4)))
    else:
        samp_Rs = np.zeros_like(samp_MK) + np.nan
    return samp_Rs
        

def _sample_Teff_from_colors(samp_GBPmag, samp_GRPmag, samp_Jmag, samp_Hmag,
                             Teff_scatter=49):
    '''Use the relation from Mann+2015 (table 2)'''
    a, b, c, d, e, f, g = 3.172, -2.475, 1.082, -.2231, .01738, .08776, -.04355
    pG = np.poly1d((e,d,c,b,a))
    p2 = np.poly1d((g,f,0))
    samp_Teff = 35e2 * (pG(samp_GBPmag-samp_GRPmag) + p2(samp_Jmag-samp_Hmag)) \
                + np.random.normal(0, Teff_scatter, samp_Jmag.size)
    return samp_Teff


def _sample_Ms_from_MK(samp_MK):
    '''Use relation from Benedict+2016'''
    if (np.median(samp_MK) > 4.6) & (np.median(samp_MK) < 9.8):
        c0 = np.random.normal(.2311, 4e-4, samp_MK.size)
        c1 = np.random.normal(-.1352, 7e-4, samp_MK.size)
        c2 = np.random.normal(.04, 5e-4, samp_MK.size)
        c3 = np.random.normal(.0038, 2e-4, samp_MK.size)
        c4 = np.random.normal(-.0032, 1e-4, samp_MK.size)
        samp_MK_tmp = np.copy(samp_MK)
        #samp_MK_tmp[(samp_MK<=4.6) | (samp_MK>10)] = np.nan
        #samp_MK_tmp[samp_MK>=10] = np.nan
        dMK = samp_MK_tmp - 7.5
        samp_Ms = c0 + c1*dMK + c2*dMK**2 + c3*dMK**3 + c4*dMK**4
    else:
        samp_Ms = np.zeros_like(samp_MK) + np.nan
    return samp_Ms


def _compute_logg(Ms, Rs):
    G = 6.67e-11
    return np.log10(G*rvs.Msun2kg(Ms)*1e2 / rvs.Rsun2m(Rs)**2)


def _compute_percentiles(samples):
    v = np.percentile(samples, (16,50,84))
    return v[1], v[2]-v[1], v[1]-v[0]

In [222]:
nanarr = np.repeat(np.nan, N)
isMdwarf = np.zeros(N, dtype=bool)
BCKs, eBCKs = np.copy(nanarr), np.copy(nanarr)
MKs, ehiMKs, eloMKs = np.copy(nanarr), np.copy(nanarr), np.copy(nanarr)
Rss2, ehiRss2, eloRss2 = np.copy(nanarr), np.copy(nanarr), np.copy(nanarr)
Teffs2, ehiTeffs2, eloTeffs2 = np.copy(nanarr), np.copy(nanarr), np.copy(nanarr)
Mss2, ehiMss2, eloMss2 = np.copy(nanarr), np.copy(nanarr), np.copy(nanarr)
loggs2, ehiloggs2, elologgs2 = np.copy(nanarr), np.copy(nanarr), np.copy(nanarr)
for i in range(N):
    
    if hasGAIA[i]:
        # sample input parameters
        samp_Jmag = np.random.randn(Nsamp)*eJmag[i] + Jmag[i]
        samp_Hmag = np.random.randn(Nsamp)*eHmag[i] + Hmag[i]
        samp_Kmag = np.random.randn(Nsamp)*eKmag[i] + Kmag[i]
        samp_GBPmag = np.random.randn(Nsamp)*eGBPmags[i] + GBPmags[i]
        samp_GRPmag = np.random.randn(Nsamp)*eGRPmags[i] + GRPmags[i]
        samp_AK = np.random.randn(Nsamp)*eAKs[i] + AKs[i]
        _,_,samp_mu = estimate_PDF.get_samples_from_percentiles(mus[i], ehimus[i], elomus[i], Nsamp=Nsamp)

        # compute new effective temperature
        samp_Teff = _sample_Teff_from_colors(samp_GBPmag, samp_GRPmag, samp_Jmag, samp_Hmag)
        Teffs2[i], ehiTeffs2[i], eloTeffs2[i] = _compute_percentiles(samp_Teff)
        
        # compute MK to see if we have an M dwarf
        samp_MK = samp_Kmag - samp_mu - samp_AK
        MKs[i], ehiMKs[i], eloMKs[i] = _compute_percentiles(samp_MK)
        isMdwarf[i] = (MKs[i] > 4.6) & (MKs[i] < 9.8)
        
        # compute bolometric corrections for non-M dwarfs and update stellar parameters
        if not isMdwarf[i]:
            BCKs[i], eBCKs[i] = _interpolate_BCK(Teffs2[i], ehiTeffs2[i], logg[i], FeH[i], AVs[i])
            samp_BCK = np.random.randn(Nsamp)*BCKs[i] + eBCKs[i]
            samp_Rs = _sample_Rs_from_MK_Kdwarfs(samp_MK, samp_BCK, samp_Teff)
            samp_Ms = np.zeros_like(samp_Rs) + np.nan  # TEMP
            
        # M dwarf parameters
        else:
            samp_Rs = _sample_Rs_from_MK_Mdwarfs(samp_MK)
            samp_Ms = _sample_Ms_from_MK(samp_MK)

        # update stellar parameters
        Rss2[i], ehiRss2[i], eloRss2[i] = _compute_percentiles(samp_Rs)
        Mss2[i], ehiMss2[i], eloMss2[i] = _compute_percentiles(samp_Ms)
        loggs2[i], ehiloggs2[i], elologgs2[i] = _compute_percentiles(_compute_logg(samp_Ms, samp_Rs))

7.05847690083 7.08141480611 7.1076609344
12.3807891615 13.1765365316 14.1149503971
12.4261023561 13.1765435214 14.1557528886
7.30476594578 7.33093949665 7.35652237789
6.58107577195 6.59251315602 6.60422830541
7.19719615422 7.21246698415 7.22570514989
5.25902825002 5.26714255348 5.27525372448
8.50260245639 8.54219818801 8.58031780518
9.30732929746 9.35835160544 9.40714631302
8.61530189485 8.65720314913 8.70198657525
8.61350981278 8.65770896115 8.70375848635
8.71946761948 8.75959761437 8.79826512726
6.73429522538 6.7437782093 6.75308147984
9.03227625559 9.08811255678 9.13925644888
8.60109551334 8.63105757527 8.6613269325
5.90067517267 5.90637494372 5.91168728144
8.15275613359 8.18803905172 8.22014194006
8.15149354445 8.18402438894 8.21729364413
8.15290389749 8.18460332662 8.22261949115
8.15358100631 8.18931667084 8.21732822641
7.95313052833 7.97218709117 7.99026898332
7.95519925643 7.97267595586 7.99245685677
8.81026028341 8.85662971131 8.90269447723
8.80946973458 8.85523557788 8.9035785

6.16937957261 6.17728140948 6.18547592502
6.16955655774 6.17682894957 6.18441710515
6.16937338594 6.17663415636 6.18479134589
8.11357004831 8.14330619673 8.17451591794
7.109603793 7.1386956861 7.16550017963
8.69091279612 8.71413492125 8.73861745713
8.83386333097 8.87175408031 8.90543409971
8.27981119431 8.31543734188 8.35184695949
8.27760464361 8.31254271441 8.34966344385
8.2799866578 8.31543888299 8.35120315684
8.27924598057 8.3132840064 8.35042925585
7.11249506835 7.13231386857 7.15421263042
7.11273343835 7.13324394381 7.15308395293
7.11271938273 7.13412281123 7.15506416002
6.4476114116 6.45352105074 6.45867896997
7.30618982785 7.33998865808 7.380552653
6.37701980862 6.38496039764 6.39290475456
8.73672368494 8.78564751986 8.83454198293
8.73279034915 8.78273246954 8.83053827156
7.94213219654 7.9720131307 8.00140831351
6.1940210945 6.20556259611 6.21710238497
6.31718633936 6.3291245939 6.3399727503
6.31749777253 6.32886550893 6.33950167007
9.56600375661 9.63088982468 9.69670030459
6.05

## save parameters to a catalog object

In [261]:
self = catalog_object('Results/Keplerlowmassstars_withplanets_GAIA_v1')
self.Nstars_withplanets = N
self.hasGAIA, self.isMdwarf = hasGAIA, isMdwarf
self.N_hasGAIA, self.N_isMdwarf = self.hasGAIA.sum(), self.isMdwarf.sum()

# positions
self.ras, self.decs = ras, decs
self.ls, self.bs = ls, bs

# photometry
self.GBPmags, self.e_GBPmags = GBPmags, eGBPmags
self.GRPmags, self.e_GRPmags = GRPmags, eGRPmags
self.Jmags, self.e_Jmags = Jmag, eJmag
self.Hmags, self.e_Hmags = Hmag, eHmag
self.Kmags, self.e_Kmags = Kmag, eKmag

# GAIA distances
self.dists, self.ehi_dists, self.elo_dists = dists, ehidists, elodists
self.mus, self.ehi_mus, self.elo_mus = mus, ehimus, elomus
self.MKs, self.ehi_MKs, self.elo_MKs = MKs, ehiMKs, eloMKs
self.AKs, self.e_AKs, self.AVs = AKs, eAKs, AVs

# updated stellar parameters
self.BCKs, self.e_BCKs = BCKs, eBCKs
self.Teffs2, self.ehi_Teffs2, self.elo_Teffs2 = Teffs2, ehiTeffs2, eloTeffs2
self.Rss2, self.ehi_Rss2, self.elo_Rss2 = Rss2, ehiRss2, eloRss2
self.Mss2, self.ehi_Mss2, self.elo_Mss2 = Mss2, ehiMss2, eloMss2
self.loggs2, self.ehi_loggs2, self.elo_loggs2 = loggs2, ehiloggs2, elologgs2

# add previous stellar parameters
self.Teffs1, self.ehi_Teffs1, self.elo_Teffs1 = Teff, ehiTeff, abs(eloTeff)
self.loggs1, self.ehi_loggs1, self.elo_loggs1 = logg, ehilogg, abs(elologg)
self.FeH1, self.ehi_FeH1, self.elo_FeH1 = FeH, ehiFeH, abs(eloFeH)
self.Rss1, self.ehi_Rss1, self.elo_Rss1 = Rss, ehiRs, abs(eloRs)
self.Mss1, self.ehi_Mss1, self.elo_Mss1 = Mss, ehiMs, abs(eloMs)

# planet parameters
self.Ps, self.ehi_Ps, self.elo_Ps = Ps, ehiP, abs(eloP)
self.rpRs, self.ehi_rpRs, self.elo_rpRs = rpRs, ehirpRs, abs(elorpRs)
self.rps1, self.ehi_rps1, self.elo_rps1 = rps, ehirp, abs(elorp)

# update planet radii
self.rps2, self.ehi_rps2, self.elo_rps2 = np.zeros(N), np.zeros(N), np.zeros(N)
for i in range(N):
    _,_,samp_rpRs = estimate_PDF.get_samples_from_percentiles(self.rpRs[i], self.ehi_rpRs[i], self.elo_rpRs[i], Nsamp=Nsamp)
    _,_,samp_Rs = estimate_PDF.get_samples_from_percentiles(self.Rss2[i], self.ehi_Rss2[i], self.elo_Rss2[i], Nsamp=Nsamp)
    samp_rp = rvs.m2Rearth(rvs.Rsun2m(samp_rpRs*samp_Rs))
    self.rps2[i], self.ehi_rps2[i], self.elo_rps2[i] = _compute_percentiles(samp_rp)
    
self._pickleobject()

0.104287 0.105209 0.105736
0.553013318407 0.571573392458 0.590885045229
0.031692 0.032537 0.033637
4.03918466889 10.5471790716 27.6444038902
0.026029 0.027927 0.028898
3.93502049171 10.7906437517 26.7526589168
0.033643 0.035061 0.035931
0.509324461547 0.525634828228 0.541946921671
0.030067 0.032227 0.036794
0.711229055306 1.61529530472 3.73455217171
0.025558 0.026906 0.030987
0.654453869488 0.675070047901 0.695086585761
0.01792 0.018884 0.025145
nan nan nan


AssertionError: 

In [263]:
hasGAIA[i]

False